<a href="https://colab.research.google.com/github/firdyannnn/Tugas-Kelompok/blob/main/TUGAS_KELOMPOK_2_ORANG_DEEP_LEARNING_LANJUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers[torch] datasets torch scikit-learn pandas numpy

In [2]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Gunakan GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Menggunakan perangkat: {device}")

MODEL_NAME = "indobenchmark/indobert-base-p2"
MAX_LENGTH = 64
BATCH_SIZE = 8
EPOCHS = 5


Menggunakan perangkat: cuda


In [5]:
kalimat_positif = [
    "Saya sangat puas dengan kualitas produk ini.", "Layanan pelanggan sangat ramah dan membantu.",
    "Aplikasi ini sangat cepat dan mudah digunakan.", "Terima kasih, pengirimannya sangat cepat.",
    "Sangat direkomendasikan untuk dibeli!", "Kualitas bahan sangat premium dan nyaman.",
    "Saya merasa senang berbelanja di sini.", "Desainnya sangat modern dan elegan.",
    "Fitur-fiturnya sangat inovatif.", "Harga sebanding dengan kualitas yang didapat.",
    "Packing rapi dan aman sampai tujuan.", "Barang asli original 100%.",
    "Seller sangat responsif dalam menjawab chat.", "Sangat membantu pekerjaan sehari-hari saya.",
    "Kualitas suara sangat jernih dan bass mantap.", "Baterai sangat awet dipakai seharian.",
    "Teksturnya lembut dan tidak lengket di kulit.", "Warna yang datang sesuai dengan pesanan.",
    "Proses pengembalian dana sangat mudah.", "Promonya sangat menarik dan menguntungkan.",
    "Tampilan UI sangat bersih dan intuitif.", "Sangat nyaman dipakai untuk olahraga.",
    "Kamera hasilnya tajam dan jernih sekali.", "Aplikasi ini sangat ringan di HP kentang.",
    "Update terbarunya sangat stabil.", "Instalasinya sangat praktis dan cepat.",
    "Menu makanannya sangat lezat dan segar.", "Tempatnya sangat bersih dan nyaman.",
    "Stafnya sangat profesional dan cekatan.", "Bonusnya sangat banyak dan bermanfaat.",
    "Sangat suka dengan wanginya yang tahan lama.", "Jahitannya sangat rapi dan kuat.",
    "Layar sangat jernih dan responsif.", "Sinyalnya sangat stabil di daerah saya.",
    "Cocok banget buat kado ulang tahun.", "Materialnya kokoh dan tidak ringkih.",
    "Pengalaman berbelanja yang luar biasa.", "Barang sampai lebih cepat dari estimasi.",
    "Sangat puas dengan garansi yang diberikan.", "Fitur keamanan sangat terjamin.",
    "Rasanya sangat otentik dan bumbunya pas.", "Harganya paling murah dibanding toko lain.",
    "Admin sangat sabar menjelaskan produk.", "Barangnya multifungsi dan praktis.",
    "Bahannya adem tidak bikin gerah.", "Warnanya sangat cantik dan estetik.",
    "Sangat efektif untuk menghilangkan noda.", "Suasananya sangat tenang dan rileks.",
    "Layanannya bintang lima banget.", "Kurirnya sangat sopan dan ramah.",
    "Sangat terbantu dengan panduan penggunaannya.", "Hasilnya langsung terlihat dalam sekali pakai.",
    "Packaging kardusnya tebal jadi aman.", "Koneksinya sangat cepat tanpa kendala.",
    "Sangat puas dengan sistem pembayarannya.", "Aplikasi ini sangat membantu produktivitas.",
    "Makanannya porsinya besar dan kenyang.", "Sangat merekomendasikan jasa ini.",
    "Kualitas sablonannya sangat bagus.", "Tidak menyesal beli di toko ini.",
    "Produk lokal tapi kualitas internasional.", "Sangat senang dapat diskon besar.",
    "Barang berfungsi dengan sangat baik.", "Fitur dark mode-nya sangat nyaman.",
    "Sangat impresif dengan teknologi barunya.", "Barang langka akhirnya ketemu di sini.",
    "Sangat ramah di kantong mahasiswa.", "Pelayanan purna jualnya jempolan.",
    "Sangat mudah mencari kategori produk.", "Review orang-orang ternyata benar bagus.",
    "Sangat membantu untuk pemula seperti saya.", "Kualitas build quality-nya solid.",
    "Sangat suka dengan packaging ramah lingkungan.", "Toko langganan yang selalu memuaskan.",
    "Sangat responsif terhadap komplain pembeli.", "Barang sesuai deskripsi 100%.",
    "Sangat efektif buat diet sehat.", "Lampu led-nya sangat terang dan fokus.",
    "Sangat stabil saat digunakan gaming.", "Proses check-out sangat simpel.",
    "Sangat suka dengan tekstur kainnya.", "Layanan antar jemputnya sangat disiplin.",
    "Sangat puas dengan update software-nya.", "Hasil fotonya sangat profesional.",
    "Sangat nyaman untuk perjalanan jauh.", "Bumbunya meresap sampai ke dalam.",
    "Sangat intuitif bagi pengguna baru.", "Stoknya selalu tersedia dan lengkap.",
    "Sangat suka dengan aromaterapinya.", "Barangnya awet sudah setahun pakai.",
    "Sangat membantu manajemen waktu saya.", "Bahannya stretch dan nyaman banget.",
    "Sangat terkesan dengan keramahan tim.", "Toko ini sangat amanah dan jujur.",
    "Sangat puas dengan pengiriman sameday.", "Ukuran pas sesuai chart yang ada.",
    "Sangat suka dengan detail produknya.", "Layanan cloud-nya sangat andal.",
    "Sangat puas, sukses terus buat tokonya!", "Maju terus karya anak bangsa."
]

kalimat_negatif = [
    "Barang yang datang rusak dan tidak berfungsi.", "Sangat kecewa dengan pelayanan toko ini.",
    "Pengiriman sangat lambat, sudah seminggu belum sampai.", "Kualitasnya jauh di bawah ekspektasi saya.",
    "Ukurannya tidak sesuai dengan deskripsi.", "Admin sangat lambat merespon chat.",
    "Aplikasi sering sekali error dan keluar sendiri.", "Bahannya tipis dan mudah robek.",
    "Sangat menyesal membeli produk ini.", "Harganya terlalu mahal untuk barang seperti ini.",
    "Barang tidak asli alias palsu.", "Warna yang dikirim salah semua.",
    "Respon penjual sangat kasar dan tidak sopan.", "Packaging sangat buruk, kotak hancur.",
    "Baterai cepat panas dan boros sekali.", "Suaranya pecah kalau volume tinggi.",
    "Sangat sulit untuk klaim garansi di sini.", "Aplikasi ini sangat berat dan lemot.",
    "Update terbaru malah bikin HP lag.", "Iklannya terlalu banyak dan mengganggu.",
    "Barang bekas dijual dengan harga baru.", "Sangat tidak direkomendasikan.",
    "Kapok belanja di toko ini lagi.", "Deskripsi produk sangat menyesatkan.",
    "Tombolnya keras dan susah ditekan.", "Baunya sangat menyengat dan tidak enak.",
    "Kamera hasilnya buram dan banyak noise.", "Sinyal sering hilang saat pakai alat ini.",
    "Bahannya kasar dan panas di kulit.", "Kurirnya tidak sopan dan main taruh saja.",
    "Uang tidak kembali padahal barang diretur.", "Isi paket tidak lengkap, ada yang kurang.",
    "Rasa makanannya hambar dan tidak segar.", "Tempatnya kotor dan tidak terawat.",
    "Stafnya tidak menguasai produk sama sekali.", "Sangat kecewa, barang tidak sesuai foto.",
    "Jahitannya berantakan dan banyak benang lepas.", "Layar ada dead pixel-nya.",
    "Koneksi sering putus secara tiba-tiba.", "Sistem pembayarannya sangat ribet.",
    "Aplikasi ini sangat membingungkan navigasinya.", "Porsinya sedikit tapi mahal banget.",
    "Sablonannya mudah mengelupas setelah dicuci.", "Barang baru dipakai sehari sudah rusak.",
    "Produk lokal tapi kualitasnya sangat buruk.", "Diskonnya cuma bohong-bohongan.",
    "Fitur yang dijanjikan tidak ada di aplikasi.", "Sangat lambat memproses pesanan.",
    "Materialnya plastik murahan dan ringkih.", "Pengiriman lewat dari batas waktu estimasi.",
    "Admin malah marah-marah saat dikritik.", "Barang datang dalam keadaan kotor.",
    "Sangat menyesal sudah bayar mahal-mahal.", "Bahannya gampang kusut dan luntur.",
    "Ukuran XL tapi kecil seperti ukuran M.", "Layanan purna jualnya sangat parah.",
    "Sangat sulit mencari menu bantuan.", "Review negatif saya dihapus oleh admin.",
    "Sangat tidak cocok untuk pemula.", "Build quality-nya sangat mengecewakan.",
    "Packaging plastiknya tipis mudah sobek.", "Toko tidak bertanggung jawab atas kerusakan.",
    "Sangat lambat merespon keluhan pelanggan.", "Barang beda jauh dengan yang di iklan.",
    "Efek sampingnya malah bikin kulit iritasi.", "Lampu led-nya redup tidak terang.",
    "Sering crash saat main game berat.", "Proses pembatalan sangat dipersulit.",
    "Tekstur kainnya kasar seperti saringan.", "Layanan antar jemput sering telat.",
    "Software-nya banyak bug yang belum diperbaiki.", "Hasil cetakannya pudar dan tidak tajam.",
    "Sangat tidak nyaman untuk diduduki lama.", "Bumbunya terlalu asin dan tidak enak.",
    "User experience-nya sangat buruk.", "Stok kosong tapi di sistem dibilang ada.",
    "Aromanya bikin pusing dan mual.", "Barang baru pakai sebulan sudah mati total.",
    "Aplikasi ini boros kuota internet.", "Bahannya kaku dan tidak nyaman bergerak.",
    "Tim teknisnya tidak membantu sama sekali.", "Toko ini sangat tidak jujur.",
    "Layanan pengiriman sameday tapi 3 hari sampai.", "Chart ukuran sangat tidak akurat.",
    "Detail produk sangat tidak jelas.", "Server-nya sering down dan tidak bisa login.",
    "Sangat tidak puas, tolong diperbaiki!", "Jangan beli di sini kalau tidak mau kecewa.",
    "Penipuan, barang tidak pernah dikirim.", "Kualitas sangat mengecewakan pelanggan.",
    "Toko paling buruk yang pernah saya temui.", "Mengecewakan sekali kualitas barangnya.",
    "Sangat buruk kinerjanya.", "Pelayanan lambat dan tidak profesional.",
    "Barang cacat tetap dikirim.", "Sangat rugi beli di sini.",
    "Respon bot terus, tidak ada manusia.", "Tidak ada solusi atas masalah saya."
]

kalimat_netral = [
    "Barang sampai dalam kondisi baik dan standar.", "Warna bajunya biru tua sesuai foto.",
    "Saya sedang mencoba menggunakan aplikasi ini.", "Paket diterima pada siang hari tadi.",
    "Tersedia berbagai pilihan pembayaran.", "Informasi di website cukup jelas.",
    "Isi paket terdiri dari unit dan charger.", "Lokasi tokonya berada di Jakarta Barat.",
    "Status pesanan sedang diproses oleh sistem.", "Barangnya biasa saja, sesuai dengan harganya.",
    "Buku ini berisi tentang panduan memasak.", "Acara akan dimulai pada pukul sembilan.",
    "Terdapat lima varian warna untuk model ini.", "Produk ini berbahan dasar plastik dan besi.",
    "Saya baru pertama kali mencoba brand ini.", "Kurir menghubungi saya lewat telepon.",
    "Spesifikasi lengkap ada di balik kemasan.", "Aplikasi ini memerlukan akses kamera.",
    "Berat paket sekitar dua kilogram.", "Barang dikirim dari gudang pusat.",
    "Ada panduan bahasa Indonesia di dalamnya.", "Laptop ini menggunakan sistem operasi terbaru.",
    "Ukuran dimensi produk adalah 10x20 cm.", "Warna yang tersedia hanya hitam dan putih.",
    "Toko buka mulai senin sampai jumat.", "Baterainya menggunakan tipe AA.",
    "Pemesanan bisa dilakukan lewat aplikasi.", "Status pengiriman sedang di sortir.",
    "Menu tersedia dalam format digital.", "Terdapat logo perusahaan di bagian depan.",
    "Harga belum termasuk ongkos kirim.", "Produk ini memerlukan dua baterai cadangan.",
    "Petunjuk pemasangan ada di dalam kotak.", "Kapasitas penyimpanan sebesar 128 GB.",
    "Layar monitor memiliki ukuran 24 inci.", "Kabel datanya panjangnya satu meter.",
    "Tersedia fitur pencarian di aplikasi.", "Pengguna bisa mengganti foto profil.",
    "Data disimpan di server secara otomatis.", "Pembayaran bisa menggunakan kartu kredit.",
    "Paket dibungkus menggunakan kertas kado.", "Terdapat nomor seri di bagian bawah.",
    "Layanan ini beroperasi 24 jam.", "Admin sedang tidak berada di tempat.",
    "Produk ini buatan dalam negeri.", "Tersedia pilihan pengiriman reguler.",
    "Stok barang akan diperbarui besok.", "Waktu operasional adalah jam kerja.",
    "Harganya tetap tidak ada diskon.", "Barang ini bisa digunakan oleh siapa saja.",
    "Bahan kainnya adalah katun murni.", "Tersedia ukuran S sampai dengan XXL.",
    "Lokasi penjemputan sudah sesuai titik.", "Aplikasi sedang melakukan pembaruan.",
    "Informasi harga ada di label tag.", "Paket sedang dalam perjalanan ke alamat.",
    "Sudah dikirim sejak dua jam yang lalu.", "Nomor resi bisa dilacak di website.",
    "Terdapat garansi resmi selama satu tahun.", "Sistem menggunakan verifikasi dua langkah.",
    "Warna asli mungkin sedikit berbeda.", "Produk ini memiliki berat yang ringan.",
    "Cara pakainya cukup ditekan sekali.", "Volume bisa diatur lewat tombol samping.",
    "Data pengguna dijamin kerahasiaannya.", "Terdapat instruksi perawatan kain.",
    "Barang ini dijual dalam satu paket.", "Isinya sebanyak 500 ml per botol.",
    "Terdapat saku di bagian depan baju.", "Kamera memiliki resolusi 12 megapixel.",
    "Aplikasi membutuhkan koneksi internet.", "Terdapat pilihan bahasa inggris.",
    "Layanan pelanggan tersedia via email.", "Produk ini tahan terhadap air.",
    "Daya listrik yang dibutuhkan 50 watt.", "Status pesanan sudah selesai.",
    "Riwayat transaksi bisa dilihat kembali.", "Tersedia slot kartu memori tambahan.",
    "Barang dikirim menggunakan ekspedisi.", "Penerima paket atas nama bapak budi.",
    "Terdapat lampu indikator di bagian atas.", "Aplikasi bisa diunduh secara gratis.",
    "Ukuran sepatu ini adalah nomor 42.", "Isi dalam box hanya satu unit.",
    "Harga sudah termasuk pajak PPN.", "Tersedia opsi pengiriman kargo.",
    "Informasi lebih lanjut ada di FAQ.", "Produk ini sudah mendapatkan izin.",
    "Terdapat tombol on/off di sisi kanan.", "Status pengiriman: Menuju hub.",
    "Barang ini baru saja dirilis bulan lalu.", "Warna netral seperti abu-abu tersedia.",
    "Proses pendaftaran memakan waktu 5 menit.", "Notifikasi akan dikirim ke email.",
    "Terdapat fitur filter pada pencarian.", "Bahan sepatu terbuat dari kulit sintetis.",
    "Status pesanan: Menunggu pembayaran.", "Terdapat tali tambahan dalam kemasan.",
    "Informasi kontak tertera di footer website.", "Produk ini sedang dalam masa promosi."
]

# Gabungkan menjadi dataframe Pelatihan
df_train = pd.DataFrame({
    'text': kalimat_positif + kalimat_negatif + kalimat_netral,
    'label': [0] * len(kalimat_positif) + [1] * len(kalimat_negatif) + [2] * len(kalimat_netral)
})
print(f"Dataset pelatihan siap: {len(df_train)} baris.")

Dataset pelatihan siap: 298 baris.


In [6]:
kalimat_uji = [
    # 1-50: Pertanyaan & Umum
    "Bagaimana cara mengganti kata sandi saya?", "Apakah aplikasi ini mendukung mode malam?",
    "Di mana saya bisa melihat riwayat pesanan?", "Berapa biaya pengiriman ke wilayah Papua?",
    "Apakah ada aplikasi untuk versi desktop?", "Saya ingin bertanya mengenai metode pembayaran.",
    "Kapan diskon besar berikutnya diadakan?", "Siapa pengembang aplikasi ini?",
    "Bagaimana sistem keamanan datanya?", "Bisa tolong jelaskan fitur terbaru ini?",
    "Daftar harga produk terbaru ada di mana?", "Apakah bisa bayar di tempat untuk area Jawa?",
    "Ada berapa banyak kategori produk yang tersedia?", "Tolong kirimkan panduan penggunaan lewat email.",
    "Saya lupa nomor pesanan saya.", "Apakah stoknya selalu tersedia setiap hari?",
    "Bagaimana cara menghubungi admin lewat telepon?", "Apakah ada promo untuk hari ini?",
    "Bisa jelaskan syarat dan ketentuan garansi?", "Lokasi toko fisik paling dekat di mana?",
    "Apakah bahan ini cocok untuk kulit sensitif?", "Tersedia ukuran apa saja untuk model ini?",
    "Bisakah saya mengubah alamat pengiriman sekarang?", "Bagaimana cara melakukan refund?",
    "Apakah aplikasi ini gratis selamanya?", "Dapatkah saya menggunakan kupon dua kali?",
    "Berapa lama proses verifikasi akun?", "Kenapa saya tidak bisa login hari ini?",
    "Bagaimana cara menghapus akun secara permanen?", "Apakah data saya aman di server?",
    "Apa keunggulan versi premium dibanding standar?", "Berapa kapasitas maksimal penyimpanan data?",
    "Bisa pakai kartu debit untuk transaksi ini?", "Daftar reseller resmi ada di mana?",
    "Apakah bisa dikirim ke luar negeri?", "Kapan paket saya dikirim oleh penjual?",
    "Bagaimana cara klaim poin hadiah?", "Apakah ada batasan usia penggunaan?",
    "Bisa minta foto asli produknya?", "Kenapa aplikasinya sering minta update?",
    "Apa saja pilihan kurir yang tersedia?", "Bagaimana cara membatalkan langganan?",
    "Apakah ada tutorial video penggunaannya?", "Bisa beli dalam jumlah grosir?",
    "Apakah warna di foto 100% akurat?", "Bagaimana sistem pembagian komisinya?",
    "Siapa yang bisa saya hubungi saat darurat?", "Apakah ada kantor cabangnya di Surabaya?",
    "Bisa bayar pakai aplikasi dompet digital?", "Kapan rilis versi terbaru aplikasinya?",

    # 51-150: Kehidupan & Lingkungan
    "Hari ini cuaca sangat tidak menentu di Jakarta.", "Gunung itu terlihat sangat megah dari sini.",
    "Saya sedang merencanakan liburan ke Bali.", "Buku ini menceritakan tentang sejarah kuno.",
    "Hutan hujan tropis sangat penting bagi bumi.", "Matahari terbit tepat pukul enam pagi.",
    "Suara hujan membuat saya merasa tenang.", "Sungai ini mengalir sampai ke laut Jawa.",
    "Angin bertiup kencang sejak sore tadi.", "Langit terlihat sangat cerah hari ini.",
    "Taman kota sekarang sudah mulai ramai.", "Banyak orang berolahraga di hari Minggu.",
    "Pohon-pohon di sini sudah mulai berbuah.", "Udara di pegunungan sangat menyegarkan.",
    "Anak-anak sedang bermain di halaman rumah.", "Kucing itu sedang tidur di atas kursi.",
    "Saya suka melihat pemandangan sawah hijau.", "Burung-burung berkicau di pagi hari.",
    "Suhu udara hari ini mencapai 32 derajat.", "Pesisir pantai sangat indah saat sunset.",
    "Kita harus menjaga kebersihan lingkungan kita.", "Tanaman di pot ini perlu disiram segera.",
    "Bunga mawar merah sedang mekar dengan cantik.", "Jalanan di sini sedang diperbaiki pemda.",
    "Kopi pagi ini terasa sangat nikmat.", "Sarapan adalah waktu makan paling penting.",
    "Besok saya akan pergi ke perpustakaan daerah.", "Lampu jalan sudah mulai menyala.",
    "Trotoar ini sekarang lebih luas dari sebelumnya.", "Pasar tradisional menjual banyak sayur segar.",
    "Ikan di akuarium ini sangat lincah berenang.", "Air sungai ini jernih sekali warnanya.",
    "Banyak mobil melintas di jalan tol.", "Kereta api berangkat tepat sesuai jadwal.",
    "Pesawat itu sedang terbang menuju arah utara.", "Pelabuhan ini sangat sibuk di malam hari.",
    "Terminal bis sekarang sudah lebih rapi.", "Museum ini menyimpan banyak koleksi berharga.",
    "Gedung pencakar langit menghiasi pusat kota.", "Monumen nasional menjadi ikon kota ini.",
    "Stadion olahraga dipenuhi oleh para penonton.", "Perpustakaan ini koleksi bukunya sangat lengkap.",
    "Universitas ini memiliki gedung yang megah.", "Sekolah dasar di desa itu baru direnovasi.",
    "Puskesmas melayani warga dengan penuh kesabaran.", "Kantor pos masih melayani pengiriman surat.",
    "Bank buka mulai jam delapan pagi.", "Anjungan tunai mandiri tersedia di setiap sudut.",
    "Supermarket ini menjual produk impor.", "Toko buku selalu menjadi tempat favorit saya.",
    "Kedai kopi ini punya barista yang handal.", "Restoran ini menu utamanya adalah rendang.",
    "Warung makan di pojok jalan itu sangat murah.", "Hotel ini punya kolam renang di lantai atas.",
    "Apartemen ini lokasinya sangat strategis.", "Rumah sakit pusat memiliki fasilitas lengkap.",
    "Dokter sedang memeriksa pasien di ruangannya.", "Apotek ini buka selama dua puluh empat jam.",
    "Pasar malam menjual berbagai macam kuliner.", "Taman bermain sangat disukai anak kecil.",
    "Kebun binatang punya koleksi hewan langka.", "Candi Borobudur adalah warisan dunia.",
    "Danau Toba memiliki pemandangan yang eksotis.", "Air terjun ini suaranya sangat keras.",
    "Goa ini gelap namun sangat menakjubkan.", "Pulau Komodo dihuni oleh hewan purba.",
    "Hutan pinus ini cocok untuk tempat berkemah.", "Sawah terasering terlihat sangat artistik.",
    "Petani sedang menanam padi di sawah.", "Nelayan berangkat melaut saat malam hari.",
    "Pasar ikan menyediakan banyak hasil laut.", "Tukang kebun sedang memotong rumput.",
    "Polisi sedang mengatur lalu lintas di simpang.", "Pemadam kebakaran bersiaga di kantornya.",
    "Tentara menjaga kedaulatan negara kita.", "Satpam berjaga di depan pintu masuk.",
    "Kurir sedang mengantarkan paket ke rumah.", "Supir bis mengemudi dengan sangat hati-hati.",
    "Pilot mempersiapkan pesawat sebelum terbang.", "Nahkoda kapal mengarahkan kapal ke dermaga.",
    "Masinis mengoperasikan kereta api listrik.", "Pramugari melayani penumpang dengan ramah.",
    "Pelayan restoran mencatat pesanan pelanggan.", "Kasir menghitung total belanjaan saya.",
    "Penjahit sedang membuat pola baju baru.", "Tukang bangunan menyusun batu bata.",
    "Arsitek sedang menggambar desain gedung.", "Insinyur memeriksa mesin di pabrik.",
    "Programmer sedang menulis baris kode program.", "Desainer grafis membuat logo perusahaan.",
    "Penulis sedang menyelesaikan naskah bukunya.", "Wartawan meliput berita di lapangan.",
    "Guru sedang mengajar di dalam kelas.", "Dosen memberikan kuliah di aula besar.",

    # 151-250: Teknologi & Inovasi
    "Kecerdasan buatan berkembang sangat pesat.", "Jaringan 5G mulai diterapkan secara merata.",
    "Smartphone lipat menjadi tren teknologi baru.", "Laptop dengan prosesor terbaru sangat kencang.",
    "Virtual reality memberikan pengalaman unik.", "Blockchain bisa digunakan untuk banyak hal.",
    "Internet of things menghubungkan peralatan rumah.", "Robotika membantu industri manufaktur.",
    "Kendaraan listrik lebih ramah terhadap lingkungan.", "Energi surya menjadi sumber listrik alternatif.",
    "Cloud computing memudahkan penyimpanan data.", "Cyber security sangat penting di era digital.",
    "Data science membantu pengambilan keputusan.", "Machine learning bisa memprediksi masa depan.",
    "Deep learning meniru cara kerja otak manusia.", "Metaverse menjadi ruang diskusi yang menarik.",
    "NFT memberikan hak milik digital yang sah.", "E-wallet memudahkan proses transaksi harian.",
    "Streaming video menggantikan televisi kabel.", "Podcast menjadi media informasi baru.",
    "Media sosial menghubungkan orang di dunia.", "E-commerce mengubah cara kita berbelanja.",
    "Fintech memberikan solusi keuangan modern.", "Edutech membantu proses belajar mengajar.",
    "Healthtech memudahkan konsultasi dengan dokter.", "Agrotech meningkatkan produktivitas petani.",
    "Smart city menggunakan teknologi untuk warga.", "Smart home membuat hidup lebih praktis.",
    "Drones digunakan untuk pengiriman barang.", "3D printing bisa mencetak berbagai benda.",
    "Biotechnology menghasilkan obat-obatan baru.", "Nanotechnology bekerja pada skala atom.",
    "Quantum computing memiliki kecepatan luar biasa.", "SpaceX berencana mengirim orang ke Mars.",
    "Satelit membantu komunikasi di seluruh bumi.", "Teleskop Hubble melihat galaksi yang jauh.",
    "Rover di Mars sedang mencari tanda kehidupan.", "Stasiun luar angkasa mengorbit bumi kita.",
    "Astronot berlatih keras sebelum ke luar angkasa.", "Roket diluncurkan dari pusat antariksa.",
    "Baterai lithium digunakan di hampir semua gadget.", "Layar OLED memberikan warna yang lebih tajam.",
    "Kamera mirrorless lebih ringan dari DSLR.", "Smartwatch bisa memantau detak jantung.",
    "Wireless charging menghilangkan keribetan kabel.", "Earbuds bluetooth sangat praktis dibawa.",
    "Power bank menjadi penyelamat saat bepergian.", "Hard disk eksternal menyimpan banyak film.",
    "SSD jauh lebih cepat dibanding HDD biasa.", "Router wifi memberikan koneksi internet rumah.",
    "Modem USB bisa dibawa kemana-mana.", "Printer inkjet bisa mencetak foto berkualitas.",
    "Scanner digunakan untuk digitalisasi dokumen.", "Keyboard mekanik sangat disukai gamer.",
    "Mouse gaming punya sensitivitas yang tinggi.", "Monitor curved memberikan efek imersif.",
    "Speaker aktif menghasilkan suara yang mantap.", "Headset gaming punya mikrofon yang jernih.",
    "Webcam digunakan untuk rapat daring.", "Microphone kondenser bagus untuk rekaman.",
    "Joystick digunakan untuk bermain game pesawat.", "Racing wheel membuat game balap terasa nyata.",
    "Konsol game terbaru grafisnya sangat indah.", "Handheld console bisa dimainkan di perjalanan.",
    "PC gaming punya spesifikasi yang sangat tinggi.", "Server rak digunakan oleh perusahaan besar.",
    "Mainboard menghubungkan semua komponen PC.", "VGA card menentukan performa grafis komputer.",
    "RAM yang besar membuat multitasking lancar.", "Cooling system menjaga suhu komponen tetap dingin.",
    "Power supply memberikan daya ke seluruh sistem.", "Casing PC punya desain yang beragam.",
    "Sistem operasi Windows banyak digunakan.", "Linux adalah sistem operasi open source.",
    "MacOS dikenal dengan tampilannya yang elegan.", "Android adalah sistem operasi mobile populer.",
    "iOS eksklusif hanya untuk produk Apple.", "Browser Chrome sangat cepat menjelajah web.",
    "Firefox adalah alternatif browser yang bagus.", "Microsoft Edge sekarang berbasis Chromium.",
    "Safari adalah browser bawaan perangkat Mac.", "Opera punya fitur VPN bawaan gratis.",
    "Antivirus melindungi komputer dari malware.", "Firewall menjaga keamanan jaringan internal.",
    "Encryption mengamankan pesan agar tidak dibaca.", "VPN menyembunyikan alamat IP asli kita.",
    "Cookie digunakan untuk menyimpan preferensi user.", "Cache mempercepat loading halaman website.",
    "API memungkinkan dua aplikasi berkomunikasi.", "Database menyimpan data secara terstruktur.",
    "Framework mempercepat proses pembuatan web.", "Library menyediakan fungsi-fungsi siap pakai.",
    "Compiler mengubah kode menjadi bahasa mesin.", "Debugger membantu mencari kesalahan kode.",
    "Version control mengelola perubahan kode.", "Deployment merilis aplikasi ke server.",
    "Hosting adalah tempat menyimpan file website.", "Domain adalah alamat unik sebuah website.",
    "SSL sertifikat mengamankan transaksi web.", "CMS memudahkan pembuatan konten website.",
    "LMS digunakan untuk kelas online sekolah.", "CRM membantu mengelola data pelanggan.",

    # 251-400: Gaya Hidup, Makanan, Travel & Opini
    "Saya suka minum teh hijau tanpa gula.", "Olahraga lari membuat jantung lebih sehat.",
    "Yoga membantu menenangkan pikiran dan tubuh.", "Membaca buku adalah jendela dunia.",
    "Menonton film di bioskop punya sensasi beda.", "Mendengarkan musik jazz sangat merilekskan.",
    "Bermain alat musik gitar itu sangat asyik.", "Memasak masakan sendiri jauh lebih hemat.",
    "Berkebun di halaman rumah sangat menyenangkan.", "Fotografi adalah hobi yang menangkap momen.",
    "Traveling ke tempat baru memperluas wawasan.", "Camping di hutan pinus terasa sangat damai.",
    "Mendaki gunung melatih kesabaran dan fisik.", "Berenang di laut biru sangat menyegarkan.",
    "Surfing menantang adrenalin di tengah ombak.", "Skateboarding butuh keseimbangan yang baik.",
    "Bermain sepak bola adalah olahraga tim.", "Basket membutuhkan koordinasi mata dan tangan.",
    "Bulutangkis sangat populer di Indonesia.", "Tenis lapangan adalah olahraga yang elegan.",
    "Catur mengasah strategi dan kemampuan otak.", "Video game bisa menjadi media belajar juga.",
    "Menggambar ilustrasi digital butuh kreativitas.", "Menulis jurnal harian membantu refleksi diri.",
    "Meditasi rutin bisa mengurangi tingkat stres.", "Tidur yang cukup penting untuk kesehatan.",
    "Minum air putih minimal dua liter sehari.", "Makan sayuran hijau kaya akan serat.",
    "Buah-buahan mengandung banyak vitamin alami.", "Menjaga pola makan itu investasi masa depan.",
    "Pakaian warna pastel sedang menjadi tren.", "Gaya minimalis membuat hidup lebih simpel.",
    "Dekorasi rumah gaya scandinavian sangat hits.", "Tanaman indoor mempercantik ruang tamu.",
    "Nasi goreng adalah makanan favorit sejuta umat.", "Sate ayam bumbu kacang rasanya juara.",
    "Soto Lamongan punya kuah yang sangat segar.", "Bakso urat rasanya sangat gurih dan nikmat.",
    "Gado-gado adalah salad khas Indonesia.", "Pempek Palembang punya cuka yang pedas.",
    "Rendang daging sapi bumbunya sangat meresap.", "Gudeg Jogja rasanya manis dan otentik.",
    "Mie Aceh punya rempah yang sangat kuat.", "Ayam Betutu khas Bali sangat pedas nikmat.",
    "Papeda adalah makanan pokok daerah timur.", "Es teler sangat cocok dinikmati saat panas.",
    "Kopi luwak adalah kopi termahal di dunia.", "Teh tarik punya rasa yang sangat unik.",
    "Jus alpukat dengan coklat sangat menggoda.", "Martabak manis topping keju coklat wijen.",
    "Pisang goreng kipas renyah sekali teksturnya.", "Klepon adalah jajanan pasar yang pecah di mulut.",
    "Lapis legit butuh ketelatenan saat membuatnya.", "Bika Ambon punya tekstur yang berserat.",
    "Rujak buah bumbu kacang sangat menyegarkan.", "Seblak Bandung pedasnya bikin ketagihan.",
    "Cimol dan cilok adalah jajanan favorit sd.", "Batagor ikan tenggiri bumbunya mantap.",
    "Kerak telor adalah makanan khas Betawi.", "Es pisang ijo sangat enak untuk berbuka.",
    "Tahu bulat digoreng dadakan lima ratusan.", "Nasi uduk harum santan dan bawang goreng.",
    "Bubur ayam diaduk atau tidak tetap enak.", "Ketoprak bumbu kacangnya sangat kental.",
    "Lontong sayur menu sarapan yang mengenyangkan.", "Rawon dengan kuah hitam yang khas.",
    "Opor ayam selalu ada saat hari raya.", "Kue kering nastar favorit saat Lebaran.",
    "Kastengel keju sangat gurih dan renyah.", "Putri salju manis dingin di lidah.",
    "Lidah kucing bentuknya tipis dan garing.", "Semprit bunga cantik warnanya kuning.",
    "Kue talam punya tekstur lembut dan kenyal.", "Kue cucur manis gula merah dan berserat.",
    "Onde-onde isi kacang hijau tabur wijen.", "Nagarsari isi pisang dibungkus daun pisang.",
    "Lemper ayam camilan yang sangat padat.", "Risoles ragout ayam atau isi mayonaise.",
    "Pastel goreng isi sayur dan telur.", "Tahu isi pedas atau tahu jeletot.",
    "Bakwan sayur garing dimakan pakai cabe.", "Tempe mendoan hangat bumbu kecap pedas.",
    "Pisang coklat lumer saat digigit panas.", "Kebab daging sapi dengan sayur segar.",
    "Burger dengan daging patty yang juicy.", "Pizza topping pepperoni dan keju mozarella.",
    "Spaghetti bolognese bumbu tomat yang asam.", "Fettuccine carbonara saus putih yang creamy.",
    "Lasagna lapisan pasta dengan daging cincang.", "Steak daging wagyu tingkat kematangan medium.",
    "Sushi salmon dengan wasabi yang menyengat.", "Ramen kuah kaldu tulang yang sangat gurih.",
    "Takoyaki isi gurita tabur katsuobushi.", "Okonomiyaki martabak ala jepang yang besar.",
    "Bibimbap nasi campur korea yang sehat.", "Tteokbokki kue beras saus gochujang pedas.",
    "Bulgogi daging iris tipis bumbu manis.", "Kimchi sawi putih yang difermentasi.",
    "Pad thai mie goreng khas Thailand.", "Tom yum kuah asam pedas dengan udang.",
    "Mango sticky rice pencuci mulut yang manis.", "Croissant roti prancis yang berlapis-lapis.",
    "Macaroon kue kecil warna-warni yang imut.", "Cheesecake tekstur lembut keju yang terasa.",
    "Tiramisu aroma kopi dan taburan coklat.", "Pancake dengan sirup maple dan mentega.",
    "Waffle renyah dengan es krim di atasnya.", "Gelato es krim ala italia yang padat.",
    "Sorbet buah dingin tanpa campuran susu.", "Smoothies bowl dengan topping granola.",
    "Matcha latte rasa teh hijau yang pekat.", "Hot chocolate di malam hari yang dingin.",
    "Cappuccino dengan foam susu yang tebal.", "Espresso shot kopi kecil yang sangat pahit.",
    "Latte art gambar di atas kopi yang indah.", "Americano kopi hitam tanpa gula dan susu.",
    "Mochaccino perpaduan kopi dan coklat.", "Iced coffee gula aren sedang sangat viral.",
    "Infused water dengan irisan lemon dan mint.", "Lemon tea segar diminum saat siang terik.",
    "Yoghurt drink baik untuk pencernaan tubuh.", "Milkshake strawberry rasa susu yang kental.",
    "Soda gembira campuran susu dan sirup merah.", "Es kelapa muda langsung dari batoknya.",
    "Cendol dawet gula merah dan santan gurih.", "Es doger dengan pacar cina dan tape.",
    "Bandrek minuman hangat jahe dan rempah.", "Bajigur minuman santan dan gula aren.",
    "Wedang ronde bola-bola ketan isi kacang.", "Wedang jahe menghangatkan badan saat hujan.",
    "Susu jahe enak diminum sebelum tidur.", "Kunyit asam jamu tradisional yang segar.",
    "Beras kencur jamu untuk stamina tubuh.", "Teh botol kemasan praktis dibawa-bawa.",
    "Air mineral kemasan gelas atau botol.", "Sirup melon dicampur air es segar.",
    "Es batu kristal lebih jernih dan bersih.", "Garam laut punya rasa asin yang berbeda.",
    "Gula pasir putih atau gula merah coklat.", "Minyak goreng kelapa atau minyak zaitun.",
    "Tepung terigu protein tinggi untuk roti.", "Mentega atau margarin untuk bahan kue.",
    "Susu kental manis untuk topping makanan.", "Keju cheddar parut atau keju lembaran.",
    "Coklat batang yang dilelehkan untuk saus.", "Selai kacang atau selai buah strawberry.",
    "Madu alami sebagai pemanis yang sehat.", "Saus sambal pedas atau saus tomat.",
    "Kecap manis kedelai hitam pilihan.", "Cuka apel untuk campuran salad buah.",
    "Bawang merah dan bawang putih ditumis.", "Cabai rawit kecil tapi sangat pedas.",
    "Merica bubuk menambah rasa hangat masakan.", "Ketumbar bubuk untuk bumbu ayam goreng.",
    "Kunir bubuk pemberi warna kuning alami.", "Jahe diparut untuk bumbu masak ikan.",
    "Lengkuas digeprek untuk aroma masakan.", "Sereh diikat untuk pengharum kuah soto.",
    "Daun salam dan daun jeruk purut.", "Kayu manis untuk aroma kue yang wangi.",
    "Cengkeh rempah asli dari Maluku.", "Bunga lawang bumbu rahasia masakan gulai."
]

# Membuat dataframe Pengujian
df_test = pd.DataFrame({'text': kalimat_uji})
print(f"Dataset pengujian siap: {len(df_test)} baris unik.")

Dataset pengujian siap: 408 baris unik.


In [7]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

class SentimentDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[item], dtype=torch.long) if self.labels is not None else None
        }

# Split data untuk evaluasi saat training
train_data, val_data = train_test_split(df_train, test_size=0.1, random_state=42)

train_set = SentimentDataset(train_data['text'].tolist(), train_data['label'].tolist(), tokenizer, MAX_LENGTH)
val_set = SentimentDataset(val_data['text'].tolist(), val_data['label'].tolist(), tokenizer, MAX_LENGTH)
test_set = SentimentDataset(df_test['text'].tolist(), None, tokenizer, MAX_LENGTH)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

In [14]:
from transformers import BertForSequenceClassification, TrainingArguments, Trainer

model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    # evaluation_strategy='steps', # Removed as it causes TypeError
    # save_strategy='steps', # Removed as it causes TypeError
    # load_best_model_at_end=True, # Removed as it depends on evaluation/save strategies
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
)

print("Memulai proses pelatihan...")
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Memulai proses pelatihan...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss
10,0.888800
20,0.311100
30,0.271700
40,0.354700
50,0.221100
60,0.104500
70,0.130400
80,0.006800
90,0.004300
100,0.002200


TrainOutput(global_step=170, training_loss=0.13542056212968687, metrics={'train_runtime': 58.0932, 'train_samples_per_second': 23.066, 'train_steps_per_second': 2.926, 'total_flos': 44071497469440.0, 'train_loss': 0.13542056212968687, 'epoch': 5.0})

In [17]:
print("Melakukan inferensi pada 400 data pengujian...")
# Jalankan prediksi
raw_preds = trainer.predict(test_set)
predictions = np.argmax(raw_preds.predictions, axis=1)

# Mapping hasil angka ke teks label
id2label = {0: "Positif", 1: "Negatif", 2: "Netral"}
df_test['predicted_label'] = [id2label[p] for p in predictions]

# Tampilkan Statistik Distribusi (Summary dari 400 data)
print("\n--- STATISTIK HASIL PELABELAN (400 DATA) ---")
print(df_test['predicted_label'].value_counts())

# Tampilkan 15 Sampel Hasil Pertama
print("\n--- 20 SAMPEL HASIL PERTAMA ---")
display(df_test.head(20))

# Menyimpan seluruh 400 data yang sudah dilabeli ke file CSV
output_filename = 'hasil_pelabelan_400_kalimat.csv'
df_test.to_csv(output_filename, index=False)

print(f"\nBerhasil menghasilkan {len(df_test)} kalimat berlabel.")
print(f"File lengkap telah disimpan dengan nama: '{output_filename}'")

Melakukan inferensi pada 400 data pengujian...



--- STATISTIK HASIL PELABELAN (400 DATA) ---
predicted_label
Netral     280
Positif    104
Negatif     24
Name: count, dtype: int64

--- 20 SAMPEL HASIL PERTAMA ---


,text,predicted_label
0,Bagaimana cara mengganti kata sandi saya?,Netral
1,Apakah aplikasi ini mendukung mode malam?,Netral
2,Di mana saya bisa melihat riwayat pesanan?,Netral
3,Berapa biaya pengiriman ke wilayah Papua?,Netral
4,Apakah ada aplikasi untuk versi desktop?,Netral
5,Saya ingin bertanya mengenai metode pembayaran.,Netral
6,Kapan diskon besar berikutnya diadakan?,Netral
7,Siapa pengembang aplikasi ini?,Netral
8,Bagaimana sistem keamanan datanya?,Netral
9,Bisa tolong jelaskan fitur terbaru ini?,Netral



Berhasil menghasilkan 408 kalimat berlabel.
File lengkap telah disimpan dengan nama: 'hasil_pelabelan_400_kalimat.csv'
